# IBM Analog Hardware Acceleration Kit (AIHWKIT): MVM using noise models characterized on the IBM HERMES Project Chip

Le Gallo, M., Khaddam-Aljameh, R., Stanisavljevic, M. et al. A 64-core mixed-signal in-memory compute chip based on phase-change memory for deep neural network inference. Nat Electron 6, 680–693 (2023). https://doi.org/10.1038/s41928-023-01010-1

In [ ]:
# %%
import os

import matplotlib.pyplot as plt
import numpy as np
import torch

from aihwkit.inference.noise.hermes import HermesNoiseModel
from aihwkit.inference.noise.pcm import PCMLikeNoiseModel
from aihwkit.simulator.configs import BoundManagementType, NoiseManagementType, WeightNoiseType
from aihwkit.inference.compensation.drift import PerColumnDriftCompensation
from aihwkit.simulator.presets import StandardHWATrainingPreset
from aihwkit.simulator.tiles import InferenceTile


# Generate a sparse uniform tensor
def generate_random_tensor_with_zeros(shape, sparsity=0.3):
    total_elements = torch.prod(torch.tensor(shape))
    num_zeros = int(total_elements * sparsity)

    # Generate a tensor from a uniform distribution
    tensor = torch.rand(shape) * 2 - 1.0

    # Randomly choose indices to set to zero
    zero_indices = torch.randperm(total_elements)[:num_zeros]

    # Set the chosen indices to zero
    tensor.view(-1)[zero_indices] = 0

    return tensor


# Perform an MVM with an ideal RPUConfig except for I/O quant and noise model
def perform_noisy_mvm(inputs, weights, noise_model, t_inf):
    rows, cols = weights.shape

    # Initialize an rpu config without ANY noise
    rpu_config = StandardHWATrainingPreset()
    rpu_config.forward.bound_management = BoundManagementType.NONE
    rpu_config.forward.out_bound = 0
    rpu_config.forward.w_noise = 0.00
    rpu_config.forward.w_noise_type = WeightNoiseType.NONE
    rpu_config.forward.ir_drop = 0.00
    rpu_config.forward.out_noise = 0.0
    rpu_config.forward.noise_management = NoiseManagementType.ABS_MAX

    # Now add on the RPU config the 8-bit I/O
    # and the selected noise model noise model
    rpu_config.forward.inp_res = 254.0
    rpu_config.forward.out_res = 254.0
    rpu_config.noise_model = noise_model

    analog_tile = InferenceTile(cols, rows, rpu_config)
    analog_tile.eval()
    analog_tile.set_weights(weights.T)

    # Apply all noises
    analog_tile.drift_weights(t_inference=t_inf)

    # Perform MVM
    output = analog_tile.forward(inputs)
    return output


def calculate_l2_error(y_target, y_measured):
    l2_errors = torch.norm(y_measured - y_target, dim=1) / torch.norm(y_target, dim=1)
    return l2_errors.mean()

In [ ]:
# Set seed and MVM/tile parameters
torch.manual_seed(42)
rows = 512
cols = 512
batch_size = 10_000
t_inf = [0.0, 60.0, 3600.0, 86400, 2592000.0, 31104000.0]
n_reps = 10

# Define the noise models to compare
noise_models = {
    "Standard": PCMLikeNoiseModel(g_max=25.0),
    "Hermes SD": HermesNoiseModel(num_devices=1),
    "Hermes TD": HermesNoiseModel(num_devices=2),
}

# Generate the data and perform the ideal MVM
inputs = generate_random_tensor_with_zeros((batch_size, rows), sparsity=0.5)
weights = generate_random_tensor_with_zeros((rows, cols), sparsity=0.3)
ideal_result = inputs @ weights

# Iterate over the models and perform 10 MVMs
mvm_precisions = {}
fig, ax = plt.subplots()
for model_n, model in noise_models.items():
    mean_l2_errs_per_t = []
    for t in t_inf:
        l2_errs = torch.zeros(n_reps)
        for i in range(n_reps):
            result_noisy = perform_noisy_mvm(inputs, weights, model, t_inf=t)
            l2_errs[i] = calculate_l2_error(ideal_result, result_noisy)
        mean_l2_errs_per_t.append(l2_errs.mean().detach() * 100)

    ax.plot(t_inf, mean_l2_errs_per_t, "-d", label=model_n)

    ax.legend()
    ax.set_xscale("log")
    ax.set_xlabel("Time of MVM")
    ax.set_ylabel("MVM error (%)")